In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    print(e)

In [1]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [2]:
c=0
for sent in Y_val:
    for el in sent:
        if len(el)>2:
            c+=1
print("number tagged elements", c)        
c=0
for sent in Y_val:
    for el in sent:
        if len(el)>2 and (el[0]=='B' or el[0]=='S'):
            c+=1
print("number tagged expressions", c)  
        

number tagged elements 13247
number tagged expressions 9728


In [3]:
from gensim.models import FastText

model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [4]:
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=20)

(3048106, 6991040)

In [8]:
from modelli.wrapper import Sequence
import tensorflow as tf

lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)
lstm.model.call((tf.keras.Input(shape=[None]),tf.keras.Input(shape=[None,None])),training=False)

In [26]:
from modelli.trainer import Trainer

trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=20, batch_size=128, verbose=1, shuffle=True)


Epoch 1/20
74/74 [==============================] - ETA: 0s - loss: 9.7659 - f1: 81.48
              precision    recall  f1-score   support

        APPL       0.88      0.82      0.85       331
        BIOP       0.57      0.67      0.62        12
        CHAR       0.74      0.73      0.74       243
      CONPRI       0.83      0.79      0.81      3239
        ENAT       0.70      0.50      0.59       197
        FEAT       0.74      0.69      0.71       487
       MACEQ       0.81      0.73      0.77       471
        MANP       0.81      0.85      0.83      1919
        MANS       0.84      0.41      0.55        51
        MATE       0.89      0.87      0.88      1795
        PARA       0.79      0.80      0.80       481
         PRO       0.83      0.86      0.85       504

   micro avg       0.83      0.80      0.81      9730
   macro avg       0.79      0.73      0.75      9730
weighted avg       0.83      0.80      0.81      9730

74/74 [==============================] - 53s 5

KeyboardInterrupt: 

In [24]:
lstm.save(model_file="model", preprocessor_file="preprocessor")

In [25]:
from modelli.wrapper import Sequence
import tensorflow as tf

lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)
lstm.load(model_file="model", preprocessor_file="preprocessor")
lstm.model.call((tf.keras.Input(shape=[None]),tf.keras.Input(shape=[None,None])),training=False)

<KerasTensor: shape=(None, None, 49) dtype=float32 (created by layer 'crf_5')>